# Showcase

## Imports

In [13]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

from datasets import InventoryDataset, ItemDataset
from network import ItemClassifier
from neurasp.neurasp import NeurASP
from utils import (
    generate_asp_program,
    generate_neural_atoms,
    get_run_number,
    load_config,
    neurasp_test,
    neurasp_train_epochs,
)

## Hyperparameters and Configuration

In [14]:
NUM_EPOCHS: int = 10
LR: float = 0.0001

In [15]:
CONFIG_PATH = Path.cwd() / "strips.yml"
TRAIN_PATH = Path.cwd() / "dataset" / "train"
VALID_PATH = Path.cwd() / "dataset" / "valid"
TEST_PATH = Path.cwd() / "dataset" / "test"
STORAGE_DIR = Path.cwd() / "results" / "neurasp"

STORAGE_DIR.mkdir(exist_ok=True, parents=True)
storage_dir = STORAGE_DIR / f"train-{get_run_number(STORAGE_DIR)}"

In [16]:
config = load_config(CONFIG_PATH)

config

StripsConfig(items=[Item(id=1, path=PosixPath('sprites/15-14.png'), name='coffee'), Item(id=2, path=PosixPath('sprites/15-11.png'), name='coffee_powder'), Item(id=3, path=PosixPath('sprites/19-5.png'), name='water'), Item(id=4, path=PosixPath('sprites/15-8.png'), name='milk'), Item(id=5, path=PosixPath('sprites/15-10.png'), name='flour'), Item(id=6, path=PosixPath('sprites/14-13.png'), name='bread'), Item(id=7, path=PosixPath('sprites/12-6.png'), name='coffee_beans')], actions=[Action(name='make_coffee', preconditions=['coffee_powder', 'water'], add_list=['coffee'], delete_list=['coffee_powder', 'water']), Action(name='bake_bread', preconditions=['water', 'flour'], add_list=['bread'], delete_list=['water', 'flour']), Action(name='wait', preconditions=[], add_list=[], delete_list=[])], time_steps=1, inventory_size=3)

## ASP Program

In [17]:
asp_program = generate_asp_program(
    actions=config.actions,
    items=config.items,
    time_steps=config.time_steps,
    inventory_size=config.inventory_size,
)

print(asp_program)

time(1..1).
have(X,0) :- init(X).
{ occur(A,T) : action(A) } = 1 :- time(T).
:- occur(A,T), pre(A,I), not have(I,T-1).
have(I,T) :- time(T), have(I,T-1), not occur(A,T) : del(A,I).
have(I,T) :- occur(A,T), add(A,I).
init_img(init_img_0).
final_img(final_img_0).
init_img(init_img_1).
final_img(final_img_1).
init_img(init_img_2).
final_img(final_img_2).
init(X) :- identify(0,I,X), init_img(I).
final(X) :- identify(0,I,X), final_img(I).
:- final(X), not have(X,1).
:- have(X,1), not final(X).
item(0).
item(1).
item(2).
item(3).
item(4).
item(5).
item(6).
item(7).
action(make_coffee).
pre(make_coffee,2).
pre(make_coffee,3).
add(make_coffee,1).
del(make_coffee,2).
del(make_coffee,3).
action(bake_bread).
pre(bake_bread,3).
pre(bake_bread,5).
add(bake_bread,6).
del(bake_bread,3).
del(bake_bread,5).
action(wait).


In [18]:
neural_atoms = generate_neural_atoms(config.items)

print(neural_atoms)

nn(identify(1,I), [0,1,2,3,4,5,6,7]) :- init_img(I).
nn(identify(1,I), [0,1,2,3,4,5,6,7]) :- final_img(I).


## Dataset

In [19]:
train_ds = InventoryDataset(TRAIN_PATH)
valid_ds = InventoryDataset(VALID_PATH)
test_ds = InventoryDataset(TEST_PATH)

item_test_ds = ItemDataset(TEST_PATH)
item_test_dl = DataLoader(item_test_ds, batch_size=4, shuffle=True)

print(f"train samples: {len(train_ds)}")
print(f"valid samples: {len(valid_ds)}")
print(f"test samples: {len(test_ds)}")

train samples: 500
valid samples: 100
test samples: 100


## Model

In [20]:
network = ItemClassifier(
    num_classes=len(config.items) + 1,  # all items including blank
)
nn_mapping = {"identify": network}
optimizers = {"identify": torch.optim.Adam(network.parameters(), lr=LR)}

network

ItemClassifier(
  (cnn): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=6272, out_features=8, bias=True)
    (1): Softmax(dim=1)
  )
)

In [21]:
model = NeurASP(
    dprogram=neural_atoms + "\n" + asp_program,
    nnMapping=nn_mapping,
    optimizers=optimizers,
)

## Training

In [22]:
neurasp_train_epochs(
    model=model,
    train_ds=train_ds,
    valid_ds=valid_ds,
    num_epochs=NUM_EPOCHS,
    storage_dir=storage_dir,
)

validation accuracy before training: 0.00 %
epoch 1/10:


100%|██████████| 500/500 [00:15<00:00, 32.54it/s]


	training accuracy: 63.20 %
	validation accuracy: 58.00 %
epoch 2/10:


100%|██████████| 500/500 [00:12<00:00, 39.48it/s]


	training accuracy: 77.40 %
	validation accuracy: 68.00 %
epoch 3/10:


...

	training accuracy: 92.60 %
	validation accuracy: 94.00 %
epoch 9/10:


100%|██████████| 500/500 [00:13<00:00, 35.98it/s]


	training accuracy: 92.60 %
	validation accuracy: 93.00 %
epoch 10/10:


100%|██████████| 500/500 [00:13<00:00, 38.28it/s]


	training accuracy: 92.00 %
	validation accuracy: 87.00 %


## Testing

In [23]:
neurasp_acc = neurasp_test(
    model=model,
    ds=test_ds,
)

print(f"NeurASP test accuracy: {neurasp_acc:.2f} %")

NeurASP test accuracy: 89.00 %


In [24]:
nn_acc, _ = model.testNN("identify", item_test_dl)

print(f"CNN test accuracy: {neurasp_acc:.2f} %")

CNN test accuracy: 89.00 %
